# Importing libraries

In [1]:
import os
import imp
import time
import glob
import torch
import nilearn
import numpy as np
from tqdm import tqdm
from nilearn import image
from scipy.ndimage.morphology import binary_dilation, binary_erosion

from utils.data_processor import *
from utils.visualization import *

%config Completer.use_jedi = False

# Creating dict with paths

In [2]:
allowed_subjects = np.load('dataset_ready_to_use/sub_with_all_data.npy', allow_pickle=True).tolist()

In [3]:
data_dict = {}
for subject in allowed_subjects:
    
    t1_brain_path = [x for x in glob.glob(f"radiologist/preprocessed_data/fmriprep/sub-{subject}/anat/*") if 'desc-preproc_T1w.nii.gz' in x and 'MNI152' in x][0]
    t2_brain_path = f"radiologist/preprocessed_data/T2/{subject}.nii"
    brain_mask_path = [x for x in glob.glob(f"radiologist/preprocessed_data/fmriprep/sub-{subject}/anat/*") if x.endswith("Asym_desc-brain_mask.nii.gz")][0]
    segmentation_path = [x for x in glob.glob(f"radiologist/preprocessed_data/fmriprep/sub-{subject}/anat/*") if 'MNI152' in x and 'GM' in x][0]
    label_path = f"radiologist/preprocessed_data/label/{subject}.nii" 
    curv_path = f"radiologist/preprocessed_data/curv/norm-{subject}.nii"
    thickness_path = f"radiologist/preprocessed_data/thickness/norm-{subject}.nii"
    sulc_path = f"radiologist/preprocessed_data/sulc/norm-{subject}.nii"
    
    data_dict[subject] = [t1_brain_path, t2_brain_path, segmentation_path, brain_mask_path,
                          label_path, curv_path, thickness_path, sulc_path]
    

# Creating data with the same size and smart sampling

In [4]:
def find_number_of_erosion(mask):
    i = 0
    tmp = mask.copy()
    while True:
        if tmp.any() == False:
            return i
        tmp = binary_erosion(tmp)
        i += 1

In [6]:
for name in ['curv', 'labels', 'segmentations', 'sulc', 't1_brains', 't2_brains', 'thickness']:
    os.makedirs(f'dataset_ready_to_use/data_full/full_new/{name}')
    
for name in ['curv', 'labels', 'segmentations', 'sulc', 't1_brains', 't2_brains', 'thickness']:
    os.makedirs(f'dataset_ready_to_use/data_only_usefull_areas/full_new/{name}')

In [7]:
number_of_brains = len(data_dict)
good_brain = nib.load('radiologist/preprocessed_data/fmriprep/sub-1/anat/sub-1_acq-T1MprageSagP2IsoOrig_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz')

for subject in tqdm(allowed_subjects):
    
    t1_brain, t2_brain, segmentation, brain_mask, label, curv, thick, sulc = data_dict[subject]
    
    nib_t1_brain = nib.load(t1_brain)
    nib_t2_brain = nib.load(t2_brain)
    nib_brain_mask = nib.load(brain_mask)
    nib_segmentation = nib.load(segmentation)
    nib_label = nib.load(label)
    nib_curv = nib.load(curv)
    nib_thick = nib.load(thick)
    nib_sulc = nib.load(sulc)
    
    nib_t1_brain = nilearn.image.resample_to_img(nib_t1_brain, good_brain)
    nib_t2_brain = nilearn.image.resample_to_img(nib_t2_brain, good_brain)
    nib_brain_mask = nilearn.image.resample_to_img(nib_brain_mask, good_brain, interpolation = 'nearest')
    arr_t1_skull_striped = nib_t1_brain.get_fdata() * nib_brain_mask.get_fdata().astype(np.int8)
    nib_t1_skull_striped = nib.Nifti1Image(arr_t1_skull_striped, nib_t1_brain.affine)
    arr_t2_skull_striped = nib_t2_brain.get_fdata() * nib_brain_mask.get_fdata().astype(np.int8)
    nib_t2_skull_striped = nib.Nifti1Image(arr_t2_skull_striped, nib_t2_brain.affine)
    
    nib_segmentation = nilearn.image.resample_to_img(nib_segmentation, good_brain, interpolation = 'nearest')
    nib_label = nilearn.image.resample_to_img(nib_label, good_brain, interpolation = 'nearest')
    nib_curv = nilearn.image.resample_to_img(nib_curv, good_brain)
    nib_thick = nilearn.image.resample_to_img(nib_thick, good_brain)
    nib_sulc = nilearn.image.resample_to_img(nib_sulc, good_brain)
    
    nib.save(nib_label, f'dataset_ready_to_use/data_full/full_new/labels/{subject}.nii')
    nib.save(nib_t1_skull_striped, f'dataset_ready_to_use/data_full/full_new/t1_brains/{subject}.nii')
    nib.save(nib_t2_skull_striped, f'dataset_ready_to_use/data_full/full_new/t2_brains/{subject}.nii')
    nib.save(nib_segmentation, f'dataset_ready_to_use/data_full/full_new/segmentations/{subject}.nii')
    nib.save(nib_curv, f'dataset_ready_to_use/data_full/full_new/curv/{subject}.nii')
    nib.save(nib_thick, f'dataset_ready_to_use/data_full/full_new/thickness/{subject}.nii')
    nib.save(nib_sulc, f'dataset_ready_to_use/data_full/full_new/sulc/{subject}.nii')

    arr_t1_brain = nib_t1_brain.get_fdata()
    arr_t2_brain = nib_t2_brain.get_fdata()
    arr_brain_mask = nib_brain_mask.get_fdata().astype(np.int8)
    arr_segmentation = nib_segmentation.get_fdata()
    arr_label = nib_label.get_fdata()
    arr_curv = nib_curv.get_fdata()
    arr_thick = nib_thick.get_fdata()
    arr_sulc = nib_sulc.get_fdata()
    
    arr_label = (arr_label>0).astype(np.int8)
    arr_t1_brain = arr_t1_brain * arr_brain_mask
    arr_t2_brain = arr_t2_brain * arr_brain_mask
    
    arr_segmentation = (arr_segmentation >= 0.5).astype(np.int8)
    grey_width = find_number_of_erosion(arr_segmentation)
    grey_dilation = int(0.8 * grey_width) 
    grey_mask_dilated = binary_dilation(arr_segmentation, iterations = grey_dilation)
    
    final_mask = np.logical_and(grey_mask_dilated, arr_brain_mask) 
    
    del grey_mask_dilated, arr_brain_mask, arr_segmentation
    
    nib.save(nib.Nifti1Image(arr_t1_brain*final_mask, nib_t1_brain.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/t1_brains/{subject}.nii')
    nib.save(nib.Nifti1Image(arr_t2_brain*final_mask, nib_t2_brain.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/t2_brains/{subject}.nii')
    nib.save(nib.Nifti1Image(arr_label*final_mask, nib_label.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/labels/{subject}.nii')
    nib.save(nib.Nifti1Image(arr_curv*final_mask, nib_curv.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/curv/{subject}.nii')
    nib.save(nib.Nifti1Image(arr_thick*final_mask, nib_thick.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/thickness/{subject}.nii')
    nib.save(nib.Nifti1Image(arr_sulc*final_mask, nib_sulc.affine), f'dataset_ready_to_use/data_only_usefull_areas/full_new/sulc/{subject}.nii')

    del nib_t1_brain, nib_t2_brain, nib_segmentation, nib_brain_mask, nib_label, nib_curv, nib_thick, nib_sulc
    
#     temple_zone_mask = np.zeros(arr_brain.shape, dtype = np.int8)
#     temple_zone_mask[:, :, :65] = 1.
    
#     final_mask_temple = np.logical_and(final_mask, temple_zone_mask)
#     final_mask_nottemple = np.logical_and(final_mask, 1 - temple_zone_mask)
    
#     del temple_zone_mask
    
#     brain_aseg_temple = arr_brain * final_mask_temple
#     brain_aseg_nottemple = arr_brain * final_mask_nottemple
    
#     label_aseg_temple = arr_label * final_mask_temple
#     label_aseg_nottemple = arr_label * final_mask_nottemple
    
#     curv_aseg_temple = arr_curv * final_mask_temple
#     curv_aseg_nottemple = arr_curv * final_mask_nottemple
    
#     thick_aseg_temple = arr_thick * final_mask_temple
#     thick_aseg_nottemple = arr_thick * final_mask_nottemple
    
#     sulc_aseg_temple = arr_sulc * final_mask_temple
#     sulc_aseg_nottemple = arr_sulc * final_mask_nottemple
    
#     del arr_brain, arr_label, arr_curv, arr_thick, arr_sulc, final_mask_temple, final_mask_nottemple 
    
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/brains/{brain_num}.npy', brain_aseg_temple)
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/brains/{brain_num}.npy', brain_aseg_nottemple)
    
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/labels/{brain_num}.npy', label_aseg_temple)
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/labels/{brain_num}.npy', label_aseg_nottemple)
    
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/curv/{brain_num}.npy', curv_aseg_temple)
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/curv/{brain_num}.npy', curv_aseg_nottemple)
    
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/thickness/{brain_num}.npy', thick_aseg_temple)
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/thickness/{brain_num}.npy', thick_aseg_nottemple)
    
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/temple/sulc/{brain_num}.npy', sulc_aseg_temple)
#     np.save(f'dataset_ready_to_use/data_only_usefull_areas/nottemple/sulc/{brain_num}.npy', sulc_aseg_nottemple)
    
#     del brain_aseg_temple, brain_aseg_nottemple,\
#         label_aseg_temple, label_aseg_nottemple,\
#         curv_aseg_temple, curv_aseg_nottemple,\
#         thick_aseg_temple, thick_aseg_nottemple,\
#         sulc_aseg_temple, sulc_aseg_nottemple

100%|██████████| 80/80 [52:45<00:00, 39.56s/it]
